In [1]:
import torch
import torch.nn as nn
from lib.architecture import Search
import math

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device

/var/folders/q6/m8k2l_j15c12q72fw85lrrqc0000gn/T/ipykernel_73320/3118330237.py:1: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")


device(type='mps')

In [3]:
N = 1000  # num_samples_per_class
D = 2  # dimensions
C = 3  # num_classes

X = torch.zeros(N * C, D).to(device)
y = torch.zeros(N * C, dtype=torch.long).to(device)

for c in range(C):
    index = 0
    t = torch.linspace(0, 1, N)
    # When c = 0 and t = 0: start of linspace
    # When c = 0 and t = 1: end of linpace
    # This inner_var is for the formula inside sin() and cos() like sin(inner_var) and cos(inner_Var)
    inner_var = torch.linspace(
        # When t = 0
        (2 * math.pi / C) * (c),
        # When t = 1
        (2 * math.pi / C) * (2 + c),
        N
    ) + torch.randn(N) * 0.2
    
    for ix in range(N * c, N * (c + 1)):
        X[ix] = t[index] * torch.FloatTensor((
            math.sin(inner_var[index]), math.cos(inner_var[index])
        ))
        y[ix] = c
        index += 1

print("Shapes:")
print("X:", tuple(X.size()))
print("y:", tuple(y.size()))

Shapes:
X: (3000, 2)
y: (3000,)


In [4]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [5]:
D = 2
H = 5
O = 3

split = Lambda(lambda x: (x[..., :H], x[..., H:]))
normalize = nn.LayerNorm(H)

encoder = nn.Sequential(
    nn.Linear(D, H),
    nn.ReLU(),
    normalize
)

search = Search(
    transition=nn.Sequential(
        nn.Linear(H, 2*H),
        nn.ReLU(),
        split,
    ),
    fitness=nn.Sequential(
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, 1),
    ),
    normalization=normalize,
    max_depth=2, 
    max_width=8, 
    beam_width=8
)

decoder = nn.Sequential(
    nn.Linear(H, 3)
)

model = nn.Sequential(
    encoder,
    search,
    decoder
)

model.to(device)

Sequential(
  (0): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
  )
  (1): Search(
    (transition): Sequential(
      (0): Linear(in_features=5, out_features=10, bias=True)
      (1): ReLU()
      (2): Lambda()
    )
    (fitness): Sequential(
      (0): Linear(in_features=5, out_features=5, bias=True)
      (1): ReLU()
      (2): Linear(in_features=5, out_features=1, bias=True)
    )
    (normalization): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
  )
  (2): Sequential(
    (0): Linear(in_features=5, out_features=3, bias=True)
  )
)

In [6]:
learning_rate = 1e-3
lambda_l2 = 1e-5

In [7]:

# nn package also has different loss functions.
# we use cross entropy loss for our classification task
criterion = torch.nn.CrossEntropyLoss()

# we use the optim package to apply
# ADAM for our parameter updates
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# e = 1.  # plotting purpose
from torch.utils.data import TensorDataset, DataLoader

with device:
    dataset = TensorDataset(X, y)
    data_loader = DataLoader(dataset, 100, shuffle=True, generator=torch.Generator(device))

    # Training
    for t in range(100):
        for batch, targets in data_loader:

            # Feed forward to get the logits
            y_pred = model(batch)
            
            # loss
            loss = criterion(y_pred, targets)
            
            # accuracy
            score, predicted = torch.max(y_pred, 1)
            acc = (targets == predicted).sum().float() / len(targets)
            
            print("[EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (t, loss.item(), acc))
            # display.clear_output(wait=False)
            
            # zero the gradients before running
            # the backward pass.
            optimizer.zero_grad()

            # clip gradient
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1e-2)
            
            # Backward pass to compute the gradient
            # of loss w.r.t our learnable params. 
            loss.backward()
            
            # Update params
            optimizer.step()

[EPOCH]: 0, [LOSS]: 1.132937, [ACCURACY]: 0.340
[EPOCH]: 0, [LOSS]: 1.259920, [ACCURACY]: 0.310
[EPOCH]: 0, [LOSS]: 1.226027, [ACCURACY]: 0.240
[EPOCH]: 0, [LOSS]: 1.330563, [ACCURACY]: 0.260
[EPOCH]: 0, [LOSS]: 1.163071, [ACCURACY]: 0.350
[EPOCH]: 0, [LOSS]: 1.202837, [ACCURACY]: 0.350
[EPOCH]: 0, [LOSS]: 1.153919, [ACCURACY]: 0.350
[EPOCH]: 0, [LOSS]: 1.250070, [ACCURACY]: 0.300
[EPOCH]: 0, [LOSS]: 1.207213, [ACCURACY]: 0.370
[EPOCH]: 0, [LOSS]: 1.128033, [ACCURACY]: 0.250
[EPOCH]: 0, [LOSS]: 1.194962, [ACCURACY]: 0.300
[EPOCH]: 0, [LOSS]: 1.175140, [ACCURACY]: 0.310
[EPOCH]: 0, [LOSS]: 1.170320, [ACCURACY]: 0.300
[EPOCH]: 0, [LOSS]: 1.198952, [ACCURACY]: 0.300
[EPOCH]: 0, [LOSS]: 1.175926, [ACCURACY]: 0.360
[EPOCH]: 0, [LOSS]: 1.156163, [ACCURACY]: 0.400
[EPOCH]: 0, [LOSS]: 1.053594, [ACCURACY]: 0.350
[EPOCH]: 0, [LOSS]: 1.146970, [ACCURACY]: 0.350
[EPOCH]: 0, [LOSS]: 1.094618, [ACCURACY]: 0.390
[EPOCH]: 0, [LOSS]: 1.182833, [ACCURACY]: 0.330
[EPOCH]: 0, [LOSS]: 1.151657, [ACCURACY]